In [1]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
for i in range(49):
    data_path = f"data/data{i}.npy"
    target_path = f"data/target{i}.npy"
    train_data = np.load(data_path)
    target_data = np.load(target_path)
    total = train_data.shape[0]
    items = np.zeros((total, 224, 224, 3), dtype=np.uint8)